*AI-Powered Nutrition Analyzer For Fitness Enthusiasts*
**Team ID PNT2022TMID43392** **--SPRINT1--**

**Importing the libraries**

In [3]:
!pip install keras
!pip install tensorflow

In [4]:
import numpy as np
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential 

**Importing Flask libraries**

In [5]:
!pip install flask

     |████████████████████████████████| 101 kB 6.7 MB/s ta 0:00:011
     |████████████████████████████████| 232 kB 57.5 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2


In [7]:
from flask import Flask, render_template,request
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import requests


**Configure ImageDataGenerator class- Image Data Augmentation**

In [8]:
train = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test=ImageDataGenerator(rescale=1./255)

In [9]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0imlvG2AYR6ttRFokvrlhzHuNScVGPFkbWNx3EyWdeyD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ainutritionanalyser-donotdelete-pr-jodnvhxqdlf9gy'
object_key = 'data.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [10]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [11]:
pwd

'/home/wsuser/work'

In [12]:
filenames = os.listdir('/home/wsuser/work/Dataset/TRAIN_SET')

**Apply Image DataGenerator Functionality To Trainset And Testset**
**Loading the data and performing data augmentation**


In [13]:
x_train = train.flow_from_directory('/home/wsuser/work/Dataset/TRAIN_SET', target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test.flow_from_directory('/home/wsuser/work/Dataset/TEST_SET', target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [14]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [15]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [16]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

**CREATING THE MODEL-->**
**Adding CNN layers**

In [17]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())



**Adding Dense Layers**

In [18]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2

In [19]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

**Compiling the model**

In [20]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

**Fitting the model-->TRAIN THE MODEL**

In [21]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_164/4010284948.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
526/526 [==============================] - 31s 59ms/step - loss: 0.1242 - accuracy: 0.9535 - val_loss: 0.1141 - val_accuracy: 0.9735
Epoch 2/20
526/526 [==============================] - 31s 59ms/step - loss: 4.4968e-04 - accuracy: 1.0000 - val_loss: 0.0651 - val_accuracy: 0.9791
Epoch 3/20
526/526 [==============================] - 31s 58ms/step - loss: 2.0678e-04 - accuracy: 1.0000 - val_loss: 0.0541 - val_accuracy: 0.9791
Epoch 4/20
526/526 [==============================] - 31s 59ms/step - loss: 0.1013 - accuracy: 0.9733 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 5/20
526/526 [==============================] - 30s 58ms/step - loss: 0.0200 - accuracy: 0.9947 - val_loss: 0.0817 - val_accuracy: 0.9706
Epoch 6/20
526/526 [==============================] - 31s 58ms/step - loss: 3.3632e-04 - accuracy: 1.0000 - val_loss: 0.0445 - val_accuracy: 0.9829
Epoch 7/20
526/526 [==============================] - 31s 58ms/step - loss: 0.0313 - accuracy: 0.9893 - val_loss: 0.0516 - v

**SAVE THE MODEL**

In [22]:
classifier.save('nutrition.h5')

In [23]:
!tar -zcvf Nutrition-analyser-model.tgz nutrition.h5

nutrition.h5


In [24]:
ls -1

Dataset/
Nutrition-analyser-model.tgz
nutrition.h5


In [25]:
pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 18.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"SeexqLh7tYnpqmWvUgLHL43ZR8TCqOwYE8Nwl13gNIv3"
}
client = APIClient(wml_credentials)


In [27]:
client = APIClient(wml_credentials)

In [28]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [29]:
space_uid = guid_from_space_name(client, 'Nutrition_analyser')
print("Space UID= " + space_uid)

Space UID= 03054542-83e6-400d-abff-089cdf60ee8d


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [40]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [41]:
model_details = client.repository.store_model(model='Nutrition-analyser-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [42]:
model_id

'544eb2a8-3e56-4bbc-aa1a-a95e5a75adc2'

In [45]:
client.repository.download(model_id,'my_model2.tar.gz')

Successfully saved model content to file: 'my_model2.tar.gz'


'/home/wsuser/work/my_model2.tar.gz'

**TESTING THE MODEL**

In [46]:
from keras.models import load_model
from keras.preprocessing import image

In [48]:
model = load_model("nutrition.h5")

In [50]:
from PIL import Image

In [51]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0imlvG2AYR6ttRFokvrlhzHuNScVGPFkbWNx3EyWdeyD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ainutritionanalyser-donotdelete-pr-jodnvhxqdlf9gy'
object_key = '1_100.jpg'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [67]:
pwd

'/home/wsuser/work'

In [68]:
model=load_model('/home/wsuser/work/nutrition.h5')

In [69]:
img=image.load_img(r'/home/wsuser/work/Dataset/TEST_SET/PINEAPPLE/1_100.jpg',target_size=(64,64))

In [70]:
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred=model.predict(x) 
y_pred=np.argmax(pred)
y_pred

3

In [71]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result = str(index[y_pred])
result

'PINEAPPLE'